In [ ]:
import numpy as np
import random
from math import cos, sin, pi ,sqrt
import rtsvg
rt = rtsvg.RACETrack()

#
# threePointCircle() - given three points, determine the circle that passes through those points
# ... does not check for collinearity ... just fails ungracefully :(
#
# Derived From:
# https://math.stackexchange.com/questions/213658/get-the-equation-of-a-circle-when-given-3-points
#
# Original Source (from that page):
# https://web.archive.org/web/20161011113446/http://www.abecedarical.com/zenosamples/zs_circle3pts.html
#
def threePointCircle(p1, p2, p3):
    x1,y1 = p1
    x2,y2 = p2
    x3,y3 = p3
    _denom_ = np.linalg.det([[x1, y1, 1.0],
                             [x2, y2, 1.0],
                             [x3, y3, 1.0]])
    _x_num_ = np.linalg.det([[x1**2 + y1**2, y1, 1.0],
                             [x2**2 + y2**2, y2, 1.0], 
                             [x3**2 + y3**2, y3, 1.0]])
    _y_num_ = np.linalg.det([[x1**2 + y1**2, x1, 1.0],
                             [x2**2 + y2**2, x2, 1.0], 
                             [x3**2 + y3**2, x3, 1.0]])    
    x0 = ( 1.0/2.0) * _x_num_/_denom_
    y0 = (-1.0/2.0) * _y_num_/_denom_
    r0 = np.sqrt((x0 - x1)**2 + (y0 - y1)**2)
    return (x0, y0, r0)

#
# counterClockwiseOrder() - arrange the pts in counter-clockwise order
# pts = [(x0,y0), (x1,y1) ...]
# c   = (x,y,r) # circle
#
# Derived From:
# https://stackoverflow.com/questions/6989100/sort-points-in-clockwise-order
#
def counterClockwiseOrder(pts, c):
    det = lambda xy0, xy1: (xy0[0] - c[0]) * (xy1[1] - c[1]) - (xy1[0] - c[0]) * (xy0[1] - c[1])
    def less(xy0, xy1):
        if xy0[0] - c[0] >= 0 and xy1[0] - c[0] < 0:  return False
        if xy0[0] - c[0] <  0 and xy1[0] - c[0] >= 0: return True
        if xy0[0] - c[0] == 0 and xy1[0] - c[0] == 0:
            if xy0[1] - c[1] >= 0 or xy1[1] - c[1] >= 0: return xy0[1] <= xy1[1]
            return xy1[1] < xy0[1]
        if det(xy0,xy1) >= 0: return True
        else:                 return False
    # insertion sort
    ordered = [pts[0]]
    for i in range(1, len(pts)):
        j = 0
        while j < len(ordered) and less(ordered[j], pts[i]): j += 1
        ordered.insert(j, pts[i])

    return ordered[::-1]

#
# should be called "pointWithinCircumcircle" # but i'll never remember what that means
# ... is this really faster than doing a square root?
#
# Derived From:
# https://en.wikipedia.org/wiki/Delaunay_triangulation
#
def pointWithinThreePointCircle(pt, p1, p2, p3):
    d          = pt
    x0, y0, r0 = threePointCircle(p1, p2, p3)
    ordered    = counterClockwiseOrder([p1, p2, p3], (x0,y0,r0))
    a, b, c    = ordered[0], ordered[1], ordered[2]
    return np.linalg.det([[a[0] - d[0], a[1] - d[1], (a[0] - d[0])**2 + (a[1] - d[1])**2],
                          [b[0] - d[0], b[1] - d[1], (b[0] - d[0])**2 + (b[1] - d[1])**2],
                          [c[0] - d[0], c[1] - d[1], (c[0] - d[0])**2 + (c[1] - d[1])**2]]) <= 0

#
# double check ordering
#
n,w,h = 10, 600, 500
c     = (300,250,200)
_svg_ = [f'<svg x="0" y="0" width="{w}" height="{h}">']
_svg_.append('<rect x="0" y="0" width="{w}" height="{h}" fill="#ffffff" />')
_svg_.append(f'<circle cx="{c[0]}" cy="{c[1]}" r="{c[2]}" stroke="#000000" fill="none" />')
pts   = []
for i in range(n):
    _angle_ = 2.0 * pi * random.random()
    x, y = c[0] + c[2] * cos(_angle_), c[1] + c[2] * sin(_angle_)
    pts.append((x,y))
    #_svg_.append(f'<text x="{x}" y="{y}" font-size="20" fill="#000000" text-anchor="middle">{i}</text>')
for i in range(20):
    x, y = random.randint(0,w), random.randint(0,h)
    if pointWithinThreePointCircle((x,y), pts[0], pts[1], pts[2]): _co_ = '#ff0000'
    else:                                                          _co_ = '#a0a0a0'
    _svg_.append(f'<circle cx="{x}" cy="{y}" r="2" fill="{_co_}" stroke="none" />')

ordered_pts = counterClockwiseOrder(pts, c)
for i in range(len(ordered_pts)):
    x,y = ordered_pts[i]
    _svg_.append(f'<text x="{x}" y="{y}" font-size="14" fill="#ff0000" text-anchor="middle">{i}</text>')

rt.tile([''.join(_svg_) + '</svg>'])

In [2]:
_n_, w, h = 10, 600, 400
pts = []
for i in range(_n_):
    x, y = random.randint(20,w-20), random.randint(20,h-20)
    pts.append((x,y))
_spacer_ = f'<svg x="0" y="0" width="20" height="{h}"> <rect x="0" y="0" width="20" height="{h}" fill="#000000" /> </svg>'

# Base Image
_svg_ = [f'<svg x="0" y="0" width="{w}" height="{h}">']
_svg_.append('<rect x="0" y="0" width="{w}" height="{h}" fill="#ffffff" />')
for i in range(_n_): _svg_.append(f'<circle cx="{pts[i][0]}" cy="{pts[i][1]}" r="2" stroke="#000000" fill="#000000" />')

# Perpendicular lines
_perps_ = []
for i in range(_n_):
    for j in range(i+1, _n_):
        x, y = (pts[i][0]+pts[j][0])/2, (pts[i][1]+pts[j][1])/2
        dx, dy = rt.unitVector((pts[i],pts[j]))
        pdx, pdy = dy, -dx
        l = max(w,y)
        _perps_.append(f'<line x1="{x+pdx*l}" y1="{y+pdy*l}" x2="{x-pdx*l}" y2="{y-pdy*l}" stroke="#000000" stroke-width="0.1" />')
        _perps_.append(f'<circle cx="{x}" cy="{y}" r="2" fill="none" stroke="#ff0000" stroke-width="0.2" />')

#rt.tile([''.join(_svg_) + '</svg>', _spacer_,''.join(_svg_) + ''.join(_perps_) + '</svg>'])

In [3]:
# Only those lines related to a specific point
_single_ = []
i  = 0
_single_.append(f'<circle cx="{pts[0][0]}" cy="{pts[0][1]}" r="4" fill="none" stroke="#ff0000" stroke-width="1" />')
for j in range(1, _n_):
    x, y = (pts[i][0]+pts[j][0])/2, (pts[i][1]+pts[j][1])/2
    dx, dy = rt.unitVector((pts[i],pts[j]))
    pdx, pdy = dy, -dx
    l = max(w,y)
    _single_.append(f'<line x1="{x+pdx*l}" y1="{y+pdy*l}" x2="{x-pdx*l}" y2="{y-pdy*l}" stroke="#000000" stroke-width="0.1" />')
    _single_.append(f'<circle cx="{x}" cy="{y}" r="2" fill="none" stroke="#ff0000" stroke-width="0.2" />')
# rt.tile([''.join(_svg_) + ''.join(_single_) + '</svg>'])

In [4]:
_circles_ = []
_centers_ = []
pts_sorted = sorted(pts)
for i in range(0, len(pts)-3):
    _circle_ = threePointCircle(pts_sorted[i], pts_sorted[i+1], pts_sorted[i+2])
    _circles_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#ff0000" stroke-width="0.1" />')
    _centers_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="2" fill="#ff0000" stroke="#ff0000" stroke-width="1" />')
# rt.tile([''.join(_svg_) + ''.join(_circles_) + '</svg>', _spacer_, ''.join(_svg_) + ''.join(_centers_) + ''.join(_single_) + '</svg>'])

In [ ]:
#
# bowyerWatson() - O(n^2) delaunay triangulation implementation
#
# Derived from:
# https://en.wikipedia.org/wiki/Bowyer%E2%80%93Watson_algorithm
#
def bowyerWatson(pts):
    # Bounds
    x_min, y_min, x_max, y_max = pts[0][0], pts[0][1], pts[0][0], pts[0][1]
    for i in range(1, len(pts)):
        x_min, y_min = min(x_min, pts[i][0]), min(y_min, pts[i][1])
        x_max, y_max = max(x_max, pts[i][0]), max(y_max, pts[i][1])

    # Super triangle
    _tip_                 = ((x_min+x_max)/2.0, (y_min - (y_max-y_min)))
    _y_base_              = y_max + (y_max-y_min)
    _x0_base_, _x1_base_  = x_min - (x_max-x_min), x_max + (x_max-x_min)
    super_triangle        = (_tip_, (_x0_base_, _y_base_), (_x1_base_, _y_base_))

    # Triangulation
    triangulation = set()
    triangulation.add(super_triangle)

    # Main loop
    for point in pts:
        # Find triangles that contain this point in their circumcircles
        bad_triangles = set()
        for triangle in triangulation:
            xy0, xy1, xy2 = triangle
            if pointWithinThreePointCircle(point, xy0, xy1, xy2):
                bad_triangles.add(triangle)
        # Normalize an edge
        def normEdge(p0, p1):
            if   p0[0] < p1[0]: return (p0, p1)
            elif p0[0] > p1[0]: return (p1, p0)
            elif p0[1] < p1[1]: return (p0, p1)
            else:               return (p1, p0)
        # Determine which edges are unique by filling in the edge lookup w/ points to their associated triangles
        edge_lu = {}
        for triangle in bad_triangles:
            xy0, xy1, xy2 = triangle
            e0, e1, e2 = normEdge(xy0, xy1), normEdge(xy1, xy2), normEdge(xy2, xy0)
            if e0 not in edge_lu: edge_lu[e0] = set()
            if e1 not in edge_lu: edge_lu[e1] = set()
            if e2 not in edge_lu: edge_lu[e2] = set()
            edge_lu[e0].add(triangle), edge_lu[e1].add(triangle), edge_lu[e2].add(triangle)

        # Construct a polygon of the unique edges
        polygon = set()
        for edge in edge_lu:
            if len(edge_lu[edge]) == 1: polygon.add(edge)
        
        # Remove all bad triangles from the triangulation
        triangulation = triangulation - bad_triangles

        # For each edge in the polygon, add a new triangle
        for edge in polygon:
            xy0, xy1 = edge
            triangulation.add((xy0, xy1, point))
    
    # Remove any triangles that had a vertex from the super triangle
    to_remove = set()
    for triangle in triangulation:
        if triangle[0] in super_triangle or triangle[1] in super_triangle or triangle[2] in super_triangle:
            to_remove.add(triangle)
    triangulation = triangulation - to_remove
    
    return triangulation

# See if it works...
_triangulation_ = bowyerWatson(pts)
_triangles_ = []
for t in _triangulation_:
    _triangles_.append(f'<polygon points="{t[0][0]},{t[0][1]} {t[1][0]},{t[1][1]} {t[2][0]},{t[2][1]}" fill="none" stroke="#ff0000" />')

rt.tile([''.join(_svg_) + ''.join(_triangles_) + '</svg>'])